In [1]:
%reload_ext cairo_jupyter

In [3]:
import cairo

from io import BytesIO
from IPython.display import Image, SVG, display
from IPython.display import display as _display

FORMAT_PS = 'ps'
FORMAT_PDF = 'pdf'
FORMAT_PNG = 'png'
FORMAT_SVG = 'svg'
FORMAT_SVG_XML = 'svg_xml'

# SURFACE_CLASSES = {
#     FORMAT_PS: cairo.PDFSurface,
    
# }

VALID_FORMATS = [FORMAT_PNG, FORMAT_SVG, FORMAT_SVG_XML]

class BufferedCanvas:
    """
    Uses a RecordingSurface to keep track of the draw state on the current page,
    and allowing it to be copied to various outputs as needed.
    """
    def __init__(self, extents=None):
        """
        :param dimensions: Extents of recording surface, or None for unbounded.
        """
        self.surface = cairo.RecordingSurface(cairo.CONTENT_COLOR_ALPHA, 
                                              extents
                                             ) 
    
    def _copy_to_surface(self, f, surface_t=cairo.ImageSurface, dimensions=None):
        if dimensions is None:
            extents = self.surface.ink_extents()
            dimensions = extents[2], extents[3]
        
        surface = surface_t(f, *dimensions)
        
        # Draw self.surface onto surface
        cr = cairo.Context(surface)
        cr.set_source_surface(self.surface)
        cr.paint()
        del cr

    def _to_svg(self, f, dimensions=None):
        surface = self._draw_to_surface(f, cairo.SVGSurface, dimensions)

        # Output SVG surface to file or buffer
        surface.finish()
    
    def _to_png(self, f, dimensions=None):
        """
        
        """
        surface = self._draw_to_surface(f, cairo.ImageSurface, dimensions)
        surface.write_to_png(f)


#         if dimensions is None:
#             extents = self.surface.ink_extents()
#             dimensions = int(extents[2]), int(extents[3])

#         surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, *dimensions)
        
#         # Draw self.surface onto surface
#         cr = cairo.Context(surface)
#         cr.set_source_surface(self.surface)
#         cr.paint()
#         del cr
        
        
    def save(self, io, format=FORMAT_PNG, *args, **kwargs):
        # TODO - is this needed, can it be used with display() ... ?
        if format == FORMAT_PNG:
            self._save_png(io, *args, **kwargs)
        elif format in (FORMAT_SVG, FORMAT_SVG_XML):
            self._save_svg(io, *args, **kwargs)
        else:
            raise ValueError("Format must be FORMAT_PNG, FORMAT_SVG or FORMAT_SVG_XML")

    def display(self, format=FORMAT_PNG, *args, **kwargs):
        with BytesIO() as io:
            if format == FORMAT_PNG:
                self._save_png(io, *args, **kwargs)
                _display(Image(data=io.getvalue()))
            elif format == FORMAT_SVG:
                self._save_svg(io, *args, **kwargs)
                _display(SVG(io.getvalue()))
            elif format == FORMAT_SVG_XML:
                try:
                    from display_xml import XML
                except ImportError:
                    raise ImportError('Install display_xml to output as XML:  $ pip3 install display_xml')

                self.save_svg(io, *args, **kwargs)
                _display(XML(io.getvalue()))
            else:
                raise ValueError("Format must be FORMAT_PNG, FORMAT_SVG or FORMAT_SVG_XML")
        
    
class ShoebotRunner:
    def __init__(self):
        self.canvas = BufferedCanvas()
    
    def run_once(self, code):
        # During first frame canvas is already cleared.
        
        # ?
        # with canvas.page():
        #    blah
        #    blah
        
        # During subsequent frames if canvas should be cleared use show_page, otherwise copy_page
        # for outputs.
        pass

# Draw on surface
canvas = BufferedCanvas()
ctx = cairo.Context(canvas.surface)
ctx.set_source_rgb(.5, .5, 1.)
ctx.rectangle(10, 10, 90, 90)
ctx.fill()

# intermediate ImageSurface 1
print("ImageSurface")
canvas.display(format=FORMAT_PNG, dimensions=(128, 128))

# intermediate SVGSurface to buffer
print("SVG to buffer")
canvas.display(format=FORMAT_SVG)
canvas.display(format=FORMAT_SVG_XML)


# continue drawing
print("continue drawing...")
ctx.set_source_rgb(1., .5, .5)
ctx.rectangle(100, 10, 90, 90)
ctx.fill()

del ctx

print("Display final surface")

canvas.surface.ink_extents()

print("Canvas display as bitmap")
canvas.display(FORMAT_PNG, dimensions=(180, 40))

print("Canvas display svg")
canvas.display(FORMAT_SVG)

ImageSurface


AttributeError: 'BufferedCanvas' object has no attribute '_save_png'